<center><h1>Singh_Vasundhara_HW2</h1></center>
<br>
<br>

Name: Vasundhara Singh
<br>
Github Username: vasu1406
<br>
USC ID: 7421414643

## 1. Combined Cycle Power Plant Data Set

### (a) Download Data

Package imports

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import statsmodels.api as statsModel
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor as KClassifier
from sklearn.preprocessing import MinMaxScaler
import math

Get the Cycle Power Plant Data Set

In [ ]:
df = pd.read_excel('../Data/CCPP/Folds5x2_pp.xlsx');

In [ ]:
df

### (b) Exploring the data

#### i. rows and columns

There are 9568 rows and 5 columns. 
The first 4 columns AT = Average temperature, V= Exhaust vacuum, AP= Ambient Pressure, RH=Relatively Humidity are the 
independent variables
The dependent variable PE = energy output. The rows represent the data/observations.

In [ ]:
print("No. of Rows =", len(df))
print("No. of Columns =",len(df.columns))
# df.info()

#### ii. pairwise scatterplots of all the varianbles

As we can see in the scatterplot below, the independent variable 'PE' seems to have some kind of linear relationship with all the dependent variables. We can also find some relations amongst the dependent variables as well. For example AT and V, and some weak relationship between AT and RH. AT has most linear relationship relationship with PE as seen in the plot.

In [ ]:
sns.pairplot(df)
plt.show()

#### iii. mean, the median, range, first and third quartiles, and interquartile ranges

In [ ]:
!pip install prettytable

from prettytable import PrettyTable

summary = PrettyTable(["Column Name", "Mean", "Median", "Range", "First Quartile", "Third Quartile","Interquartile Ranges"])
  
# Add rows
for column in df.columns:
    thirdQ=df[column].quantile(0.75)
    firstQ=df[column].quantile(0.25)
    summary.add_row([column,round(df[column].mean(),6),round(df[column].median(),6),df[column].quantile(1)-df[column].quantile(0),firstQ,thirdQ,thirdQ-firstQ])
  
print(summary)

### (c) Simple Linear Regression

I am using Cook's Distance to find out the outliers in the data. Here we identify the outliers by the data which is more than 4/n in the cooks distance. These outliers are shown in Red color in the plots. As we can see there are significant no. of outliers in each regression task, we can remove them to get better prediction.

All the independent variables have a statistically significant association with the dependent variable as all of them have a
p-value much less than 0.05.

In [ ]:
def fetch_cooks_points(cooks,x,y):
    outliersMap = dict()
    n=len(cooks)
    
    for i in range(0,n):
        if cooks[i] >= 4/n:
            outliersMap[x[i]]=y[i]
    return outliersMap

coeffMap=dict()
independentVariables = df.columns[:-1]
dependentVariables = df.columns[-1]
energyOutputColumn = df['PE']

for column in independentVariables:
    
    model = statsModel.OLS(energyOutputColumn,statsModel.add_constant(df[column])).fit()
    sns.regplot(x = column,y = dependentVariables, data = df,line_kws = {'color': 'black'})

    cooks = model.get_influence().cooks_distance
    
    
    cooks_map = fetch_cooks_points(cooks[0],df[column],df['PE'])
    keys = cooks_map.keys()
    values = cooks_map.values()

#     print(cooks_map)
    plt.scatter(keys,values,color ='red')
    plt.show()
    
    print(model.params)
    coeffMap[column] = [column,model.params[column]]
    results_summary = model.summary()
#     p_values = results_summary.tables[1]['P>|t|']
#     table = results_summary.tables[1]
#     table = pd.DataFrame(table.data)
#     p_values = table.loc[:, table.columns == 'P>|t|'].iloc[:, 0].tolist()
#     print('P Value',p_values)
    print(results_summary)

### (d) Multiple Regression

The null hypothesis states that the independent variable has null or no effect on the dependent variable. But as we see in the results below, even after fitting a multiple regression model to predict the response, the p values are much less than 0.05 and
way closer to 0, which shows all the predictors have a statistically significant association with the dependent variable. Hence
we can reject the null hypothesis for all the predictors.

In [ ]:
independentVariables = df.columns[:-1]
energyOutputColumn = df['PE']

multipleRegressionModel = statsModel.OLS(energyOutputColumn,statsModel.add_constant(df[independentVariables])).fit()
results_summary_multiple = multipleRegressionModel.summary()

print(results_summary_multiple)
print(energyOutputColumn.shape)
print(df[independentVariables].shape)

### (e) 1c Compare to 1d

1c results shows that each independent variable have statistically significant association with the dependent variable and 1d 
results shows that there is statistical significance when combined set of all the independent variables are used for prediction.

In [ ]:
multipleRegressionModel.params
for column in independentVariables:
    coeffMap[column].append(multipleRegressionModel.params[column])
for keys,values in coeffMap.items():
    print(keys,' ',values)

In [ ]:

map = pd.DataFrame.from_dict(coeffMap,orient='index',columns=['Attributes','Univariate Coefficients','Multivariate Coefficients'])

mapRows=[]
columnNames=['Attributes','Univariate Coefficients','Multivariate Coefficients']
for key,values in coeffMap.items():
    mapRows.append([key,values[1],values[2]])

pd.DataFrame(mapRows,columns = columnNames)

In [ ]:
sns.scatterplot(x='Univariate Coefficients',y='Multivariate Coefficients',data = map,hue='Attributes')
plt.title("Univariate VS Multivariate Coefficients")
plt.ylabel('Multivariate Coefficients')
plt.xlabel('Univariate Coefficients')
plt.show()

### (f) Nonlinear Association

We see some predictors have statistical significance when they have squared or cubic terms. For example- AT,AP and RH has statistical significance in linear,quadratic and cubic term as the p-value is close to 0. It's good idea to explore their non linear relationship with the dependent variable as p-values is very low, hence they have a non-linear relationship with the dependant variable. However, V is statistically insignificant when squared, as V^2 has very high p-value. Hence V^2 doesn't have a strong non-linear relationship with the dependent variable.

In [ ]:
for col in independentVariables:
    polynomial = PolynomialFeatures(3)
    polynomial_wrt_x = pd.DataFrame(polynomial.fit_transform(df[col].values.reshape(-1,1)),columns = polynomial.get_feature_names_out([col]))
    model = statsModel.OLS(endog = energyOutputColumn,exog = polynomial_wrt_x).fit()
    print('Attribute =',col)
    display(model.summary())
#     tables = model.summary().tables[1]
#     print(tables)

### (g) Interactions of Predictors

As AT X V, AT X RH, V X AP, AP X RH have very small p values which are less than 0.05. All these interaction terms have statistical significance

In [ ]:
#interaction set to true

polynomial_Interactions = PolynomialFeatures(2, interaction_only = True)
x_Interactions = df[independentVariables]
polynomial_wrt_all_terms = pd.DataFrame(polynomial_Interactions.fit_transform(x_Interactions),columns = polynomial_Interactions.get_feature_names_out(independentVariables))
model = statsModel.OLS(energyOutputColumn,polynomial_wrt_all_terms).fit()
display(model.summary())
# tables = model.summary().tables[1]
# print(tables)

### (h) Improvement

In [ ]:
polynomial_wrt_x=PolynomialFeatures(2)
polynomial = PolynomialFeatures(2)
x_Interactions = df[independentVariables]
xData = pd.DataFrame(polynomial.fit_transform(x_Interactions),columns=polynomial.get_feature_names_out(independentVariables))
trainingX, testX, trainingY, testY = tts(xData, energyOutputColumn, test_size = 0.3, random_state = 42)


In [ ]:
arr_col = ['1'] + [col for col in independentVariables]

model = statsModel.OLS(trainingY,trainingX[arr_col]).fit()
predictedYTest = model.predict(testX[arr_col])
predictedYTrain = model.predict(trainingX[arr_col])

minSquaredErrorTest = mean_squared_error(testY,predictedYTest)
minSquaredErrorTrain = mean_squared_error(trainingY,predictedYTrain)
print('Min Squared Error Test =',minSquaredErrorTest)
print('Min Squared Error Train =',minSquaredErrorTrain)

display(model.summary())
# tables = model.summary().tables[1]
# display(tables)
# print(tables)

We can see that V^2, V X AP, and V X RH have very high p-values hence they are statistically insignificant, hence I have dropped it and calculated the MSE. Removing these terms increases the MSE very slightly.

In [ ]:
#All terms and interaction_Terms

model = statsModel.OLS(trainingY,trainingX).fit()
predictedYTest = model.predict(testX)
predictedYTrain = model.predict(trainingX)

minSquaredErrorTest = mean_squared_error(testY,predictedYTest)
minSquaredErrorTrain = mean_squared_error(trainingY,predictedYTrain)
print('Min Squared Error Test =',minSquaredErrorTest)
print('Min Squared Error Train =',minSquaredErrorTrain)

display(model.summary())
# tables = model.summary().tables[1]
# print(tables)

print('\n\n')
print('After removing statistically insignificant terms, the MSE increases slightly\n\n')

# Remove all the statistically insignificant terms (V^2, VXRH and VXAP) which have p values greater than 0.05
# As we are removing some of terms, the MSE increases slightly

significantX=['1','AT','V','AP','RH','AT^2','AT V','AT RH','AP^2','AP RH','RH^2']

model = statsModel.OLS(trainingY,trainingX[significantX]).fit()
predictedYTest = model.predict(testX[significantX])
predictedYTrain = model.predict(trainingX[significantX])


minSquaredErrorTest = mean_squared_error(testY,predictedYTest)
minSquaredErrorTrain = mean_squared_error(trainingY,predictedYTrain)
print('Min Squared Error Test =',minSquaredErrorTest)
print('Min Squared Error Train =',minSquaredErrorTrain)

display(model.summary())
# tables = model.summary().tables[1]
# print(tables)

### (i) KNN

The k value for least error is around 4-5. It changes sometimes because of random slicing of data.

In [ ]:
def returnKeywithMinimumError(errorMap):
    return min(zip(errorMap.values(), errorMap.keys()))[1]

def KNearestNeighbors(trainingX,trainingY,testX,testY):
    
    errorMapTrain = dict()
    errorMapTest = dict()
    
    for k in range(1,101):
        
        kRegression = KClassifier(n_neighbors = k)
        kRegression.fit(trainingX, trainingY)
        
        predictedYTest = kRegression.predict(testX)
        predictedYTrain = kRegression.predict(trainingX)
        
        errorMapTest[1/k] = mean_squared_error(testY,predictedYTest)
        errorMapTrain[1/k] = mean_squared_error(trainingY,predictedYTrain)
    
    optimalK = returnKeywithMinimumError(errorMapTest)
    leastError = errorMapTest[optimalK]
    
    print('Most Optimal K=',1/optimalK,' with least error = ',leastError)
    
    # Plot train and test errors in terms of k

    plt.plot(*zip(*sorted(errorMapTrain.items())), label='Training Error')
    plt.plot(*zip(*sorted(errorMapTest.items())), label = 'Test Error')
    plt.xlabel('1/k')
    plt.ylabel('Error')
    plt.legend()
    plt.show()

print('\n\nRAW DATA')
KNearestNeighbors(trainingX[independentVariables],trainingY, testX[independentVariables], testY)

#Normalisation
# trainingData = trainingX[independentVariables]
# testData = testX[independentVariables]
scaler = MinMaxScaler()
trainingXNormalized = pd.DataFrame(scaler.fit_transform(trainingX))
testXNormalized = pd.DataFrame(scaler.fit_transform(testX))


print('\n\nNORMALISED DATA')
KNearestNeighbors(trainingXNormalized,trainingY,testXNormalized,testY)

### (j ) Compare KNN and Linear

Min Squared Error Test = 18.64731247858754 for Multiple Linear Regression Model with all interaction terms and non-linear terms.

FOR KNN 
RAW DATA- Most Optimal K= 4.0-5.0  with least error approximately 15.726819842563568
NORMALISED DATA
Most Optimal K= 8.0-9.0  with least error approximately 16.023740527581857


The results clearly shows that the Minimum Squared Error for KNN Regression is lesser than the Minimum Squared Error obtained in
different Linear Regression Models. Hence KNN is better suited for the data. And in case of our data, normalisation didn't work 
in favor of us and the MSE was increased.

In KNN, it is hard to interpret the effect of each term on the output. On the 
other hand we can understand the impact of each attribute on the output throught the coefficients we get in Linear Regression.


## 2. ISLR: 2.4.1

### (a) The sample size n is extremely large, and the number of predictors p is small.

In this case, a flexible statistical learning method would have better performance than an inflexible one. As the n is large, and hence the sample size is large and we would have enough information to model the relationships between predictors and output
better, even if there is a complex model. In addition to that, having a small no. of predictors means the model will not have 
'overfitting' or the curse of dimensionality which can make the performance of inflexible methods suffer.

### (b) The number of predictors p is extremely large, and the number of observations n is small.

In this case, an inflexible statistical learning method would have better performance than a flexible one. As the n is small,
and hence the sample size is small, we would not have enough information to model the relationships between predictors and
output/response accurately. If the no. of predictors is high, it makes it even more difficult to develop an accurate model. In 
such cases the model fits the noise in the data rather than fitting the relationship better, which leads to poor prediction on 
test/unseen data. Inflexible methods have lower risks of overfitting and hence would perform better in this case.

### (c) The relationship between the predictors and response is highly non-linear.

In this case, a flexible statistical learning method would have better performance than an inflexible one. The flexible methods
are proved to be better at modeling non-linear, complex relationships between predictors and response than inflexible ones.
Whereas, the inflexible methods assume the reltionships to be simple or linear and hence do not perform well when it comes to 
complex or non-linear relationships. Flexbile statistical learning methods such as Decision Trees and Non-linear regression 
are used to model such scenarios for better performance

### (d) The variance of the error terms, i.e. $σ^2$ = Var(ε), is extremely high.

When the variance of the error terms is too high, it shows that there is a lot of noise in the data which inevitably makes it 
difficult to accurately model the relationship between the predictors and response. As we discussed in the above questions 
flexible models run a higher risk of overfitting than inflexible ones. So if flexible methods are used in this case, it might 
try to fit the noise in the data leading to overfitting. Wheras, an inflexible method are less prone to overfitting and hence 
they would perform better than flexible method.

## 3. ISLR: 2.4.7

### (a) Compute the Euclidean distance between each observation and the test point, X1 = X2 = X3 = 0.

In [ ]:
def euclidean_distance_3d(point1, point2):
    x1, y1, z1 = point1
    x2, y2, z2 = point2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

point1 = (0, 0, 0)
#Observations from the questions
points = [(0, 3, 0), (2,0,0), (0,1,3), (0,1,2),(-1,0,1), (1,1,1)]

for point in points:
    distance = euclidean_distance_3d(point1, point)
    print('Euclidean Distance between', point,'& ', point1,' = ', distance)

### (b) What is our prediction with K = 1? Why?

When k=1, we are just finding the single nearest neighbor to our test point (0,0,0). And as the observation no.5 
which is (-1,0,1) is closest to our test point with a minimum distance of 1.414. So our predicted Y will be same as the Y
of closest point which is "Green".

### (c) What is our prediction with K = 3? Why?

When k=3, we are finding three nearest neighbor to our test point (0,0,0) which will include observation no.5, observation no.6
and observation no. 2 as they are the closest 3 points to our test point. So our predicted Y will be same as the Y that the 
majority of the nearest neighbors have and here two out of three neighbors have the Y = 'Red'. Hence the prediction will be
"Red".

### (d) If the Bayes decision boundary in this problem is highly non-linear, then would we expect the best value for K to be large or small? Why?

If the Bayes decision boundary is highly non-linear, we would need a model that gives better performance in case of non-linearity.The flexible statistical learning methods are better suited for modelling non-linear and complex relationships. For KNN to be more flexible, it should have a smaller value of K. As we take the average or majority of the K points, large K will lead to inflexible models and hence would not model the non-linear relationships accurately/better than small k.
Hence best k values should be small